In [1]:
import asyncio
import datetime
import os
import threading
import time
from collections import deque
import nest_asyncio

from arduino_iot_cloud import ArduinoCloudClient
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Apply nest_asyncio to allow asyncio to run within another event loop
nest_asyncio.apply()

# --- Arduino Cloud Configuration ---
DEVICE_ID = "7c085c20-b2ad-4248-88cb-fa699d451afe"
SECRET_KEY = "XqwDz6uY8og9oQzGj6NFUThHL"

# --- Data Storage ---
MAX_DATA_POINTS = 300  # Increased buffer for smoother visuals
data_history = {
    "timestamp": deque(maxlen=MAX_DATA_POINTS),
    "x": deque(maxlen=MAX_DATA_POINTS),
    "y": deque(maxlen=MAX_DATA_POINTS),
    "z": deque(maxlen=MAX_DATA_POINTS),
}

# Buffer for incomplete data points
pending_data = {"x": None, "y": None, "z": None}
data_lock = threading.Lock()
last_update_time = time.time()

# --- Arduino Client and Data Collection Logic ---
def on_data_received(client, name, value):
    """Callback to handle incoming data and update the shared deque."""
    global last_update_time
    
    with data_lock:
        if name == "py_x":
            pending_data["x"] = value
        elif name == "py_y":
            pending_data["y"] = value
        elif name == "py_z":
            pending_data["z"] = value
        
        # Only update when we have all three values
        if all(v is not None for v in pending_data.values()):
            current_time = datetime.datetime.now()
            data_history["timestamp"].append(current_time)
            data_history["x"].append(pending_data["x"])
            data_history["y"].append(pending_data["y"])
            data_history["z"].append(pending_data["z"])
            
            # Reset pending data
            pending_data["x"] = None
            pending_data["y"] = None
            pending_data["z"] = None
            
            last_update_time = time.time()

async def arduino_client_main():
    """Main async function to run the Arduino client."""
    client = ArduinoCloudClient(device_id=DEVICE_ID, username=DEVICE_ID, password=SECRET_KEY)
    
    # Register properties and their callbacks
    client.register("py_x", on_write=lambda c, v: on_data_received(c, "py_x", v))
    client.register("py_y", on_write=lambda c, v: on_data_received(c, "py_y", v))
    client.register("py_z", on_write=lambda c, v: on_data_received(c, "py_z", v))

    print("Starting Arduino Cloud client in a background thread...")
    await client.start()
    
    # Keep the client running
    while True:
        await asyncio.sleep(0.01)  # Very short sleep for responsiveness

def start_arduino_client_thread():
    """Starts the asyncio event loop for the client in a new thread."""
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(arduino_client_main())

# --- Dash Application ---
def create_dashboard_app(data_deques, title, update_interval_ms=100):
    """
    Creates and configures a Plotly Dash application for displaying real-time accelerometer data.
    """
    app = dash.Dash(__name__, title=title)
    
    app.layout = html.Div([
        html.H1(title, style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': '20px'}),
        
        # Status indicator
        html.Div(id='status-indicator', style={
            'height': '20px', 
            'width': '20px', 
            'backgroundColor': 'gray', 
            'borderRadius': '50%',
            'display': 'inline-block',
            'marginLeft': '20px'
        }),
        html.Span(id='status-text', style={'marginLeft': '10px'}),
        
        # Combined plot
        dcc.Graph(id='combined-plot', style={'height': '300px'}),
        
        # Individual axis plots in a row
        html.Div([
            dcc.Graph(id='x-plot', style={'height': '250px', 'width': '32%', 'display': 'inline-block'}),
            dcc.Graph(id='y-plot', style={'height': '250px', 'width': '32%', 'display': 'inline-block', 'marginLeft': '2%'}),
            dcc.Graph(id='z-plot', style={'height': '250px', 'width': '32%', 'display': 'inline-block', 'marginLeft': '2%'}),
        ], style={'width': '100%', 'marginTop': '20px'}),
        
        dcc.Interval(
            id='interval-component',
            interval=update_interval_ms,
            n_intervals=0
        ),
        
        # Store for tracking data updates
        dcc.Store(id='data-update-tracker', data={'last_update': 0}),
    ], style={'padding': '20px', 'fontFamily': 'Arial, sans-serif'})

    # Callback to update status indicator
    @app.callback(
        [Output('status-indicator', 'style'), Output('status-text', 'children')],
        Input('interval-component', 'n_intervals')
    )
    def update_status(n):
        current_time = time.time()
        time_diff = current_time - last_update_time
        
        status_style = {
            'height': '20px', 
            'width': '20px', 
            'borderRadius': '50%',
            'display': 'inline-block',
            'marginLeft': '20px'
        }
        
        if time_diff < 2:  # Updated in last 2 seconds
            status_style['backgroundColor'] = 'green'
            status_text = "Receiving data (updated {:.1f}s ago)".format(time_diff)
        elif time_diff < 5:
            status_style['backgroundColor'] = 'orange'
            status_text = "Slow data (updated {:.1f}s ago)".format(time_diff)
        else:
            status_style['backgroundColor'] = 'red'
            status_text = "No recent data (updated {:.1f}s ago)".format(time_diff)
            
        return status_style, status_text

    # Callback for the combined plot
    @app.callback(
        Output('combined-plot', 'figure'),
        Input('interval-component', 'n_intervals')
    )
    def update_combined_plot(n):
        with data_lock:
            timestamps = list(data_deques["timestamp"])
            x_data = list(data_deques["x"])
            y_data = list(data_deques["y"])
            z_data = list(data_deques["z"])
        
        fig = go.Figure()
        
        if timestamps and x_data:
            fig.add_trace(go.Scatter(
                x=timestamps, y=x_data,
                name='X-axis',
                line=dict(color='#e74c3c')
            ))
            
        if timestamps and y_data:
            fig.add_trace(go.Scatter(
                x=timestamps, y=y_data,
                name='Y-axis',
                line=dict(color='#2ecc71')
            ))
            
        if timestamps and z_data:
            fig.add_trace(go.Scatter(
                x=timestamps, y=z_data,
                name='Z-axis',
                line=dict(color='#3498db')
            ))
        
        fig.update_layout(
            title='Combined Accelerometer Data',
            xaxis_title='Time',
            yaxis_title='Acceleration (m/s²)',
            legend_title='Axis',
            template='plotly_white',
            hovermode='x unified',
            uirevision='constant'  # Preserves UI state during updates
        )
        
        return fig

    # Callback for X-axis plot
    @app.callback(
        Output('x-plot', 'figure'),
        Input('interval-component', 'n_intervals')
    )
    def update_x_plot(n):
        with data_lock:
            timestamps = list(data_deques["timestamp"])
            x_data = list(data_deques["x"])
        
        fig = go.Figure()
        
        if timestamps and x_data:
            fig.add_trace(go.Scatter(
                x=timestamps, y=x_data,
                name='X-axis',
                line=dict(color='#e74c3c')
            ))
        
        fig.update_layout(
            title='X-Axis Acceleration',
            xaxis_title='Time',
            yaxis_title='Acceleration (m/s²)',
            template='plotly_white',
            uirevision='constant'
        )
        
        return fig

    # Callback for Y-axis plot
    @app.callback(
        Output('y-plot', 'figure'),
        Input('interval-component', 'n_intervals')
    )
    def update_y_plot(n):
        with data_lock:
            timestamps = list(data_deques["timestamp"])
            y_data = list(data_deques["y"])
        
        fig = go.Figure()
        
        if timestamps and y_data:
            fig.add_trace(go.Scatter(
                x=timestamps, y=y_data,
                name='Y-axis',
                line=dict(color='#2ecc71')
            ))
        
        fig.update_layout(
            title='Y-Axis Acceleration',
            xaxis_title='Time',
            yaxis_title='Acceleration (m/s²)',
            template='plotly_white',
            uirevision='constant'
        )
        
        return fig

    # Callback for Z-axis plot
    @app.callback(
        Output('z-plot', 'figure'),
        Input('interval-component', 'n_intervals')
    )
    def update_z_plot(n):
        with data_lock:
            timestamps = list(data_deques["timestamp"])
            z_data = list(data_deques["z"])
        
        fig = go.Figure()
        
        if timestamps and z_data:
            fig.add_trace(go.Scatter(
                x=timestamps, y=z_data,
                name='Z-axis',
                line=dict(color='#3498db')
            ))
        
        fig.update_layout(
            title='Z-Axis Acceleration',
            xaxis_title='Time',
            yaxis_title='Acceleration (m/s²)',
            template='plotly_white',
            uirevision='constant'
        )
        
        return fig

    return app

# --- Main Application Execution ---
if __name__ == '__main__':
    # 1. Start the Arduino data collection in a daemon thread
    arduino_thread = threading.Thread(target=start_arduino_client_thread, daemon=True)
    arduino_thread.start()

    # 2. Create the Dash app
    app = create_dashboard_app(
        data_deques=data_history,
        title="SIT225 - Real-time Smartphone Accelerometer Dashboard",
        update_interval_ms=100  # Original update time
    )

    # 3. Run the Dash server
    print("Starting Dash server... Check the output for the URL.")
    app.run(debug=True, jupyter_mode="tab")

C:\Users\letan\anaconda3\envs\clean_env\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Starting Arduino Cloud client in a background thread...
Starting Dash server... Check the output for the URL.
Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

ERROR:root:task: connection_task raised exception: .
ERROR:root:task: discovery raised exception: .
